In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
import tensorflow_datasets as tfds
import numpy as np
import pickle

# Config
IMG_SIZE = 64
NUM_CLASSES = 62
BATCH_SIZE = 64
EPOCHS_INITIAL = 5
EPOCHS_FINE_TUNE = 5

# Disable mixed precision (too heavy for your CPU)
# tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Preprocessing
def preprocess(image, label):
    image = tf.image.transpose(image)  # match EMNIST orientation
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.grayscale_to_rgb(image)
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

# Load and prepare EMNIST
(ds_train, ds_test), ds_info = tfds.load('emnist/byclass', split=['train', 'test'], as_supervised=True, with_info=True)
ds_train = ds_train.take(40000)  # speed up

train_ds = ds_train.map(preprocess).shuffle(2048).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = ds_test.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# MobileNetV2 model
input_layer = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
base_model = MobileNetV2(include_top=False, weights='imagenet', input_tensor=input_layer)
base_model.trainable = False

x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.4)(x)
output = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = models.Model(inputs=input_layer, outputs=output)

# Compile
model.compile(optimizer=Adam(1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ReduceLROnPlateau(patience=2, factor=0.5, min_lr=1e-6),
    ModelCheckpoint("EMNIST_MobileNetV2_best.h5", save_best_only=True)
]

print("[INFO] Starting Stage 1 training...")
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS_INITIAL, callbacks=callbacks)

# Fine-tuning
for layer in base_model.layers[-20:]:
    layer.trainable = True

model.compile(optimizer=Adam(1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("[INFO] Starting Stage 2 fine-tuning...")
history = model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS_FINE_TUNE, callbacks=callbacks)

# Save
model.save("mobilenet_emnist_gui_model.h5")
with open('emnist_mobilenet_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

print("✅ Model and history saved.")


C:\Users\Hashtag\AppData\Local\Temp\ipykernel_4504\3433335856.py:39: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(include_top=False, weights='imagenet', input_tensor=input_layer)


[INFO] Starting Stage 1 training...
Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.4755 - loss: 2.1287

625/625 ━━━━━━━━━━━━━━━━━━━━ 153s 238ms/step - accuracy: 0.4756 - loss: 2.1278 - val_accuracy: 0.6922 - val_loss: 1.0370 - learning_rate: 0.0010
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.6874 - loss: 1.0837

625/625 ━━━━━━━━━━━━━━━━━━━━ 158s 253ms/step - accuracy: 0.6874 - loss: 1.0836 - val_accuracy: 0.7340 - val_loss: 0.8869 - learning_rate: 0.0010
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.7132 - loss: 0.9575

625/625 ━━━━━━━━━━━━━━━━━━━━ 147s 235ms/step - accuracy: 0.7132 - loss: 0.9575 - val_accuracy: 0.7423 - val_loss: 0.8565 - learning_rate: 0.0010
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.7294 - loss: 0.8941

625/625 ━━━━━━━━━━━━━━━━━━━━ 157s 252ms/step - accuracy: 0.7294 - loss: 0.8941 - val_accuracy: 0.7455 - val_loss: 0.8402 - learning_rate: 0.0010
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 153s 244ms/step - accuracy: 0.7399 - loss: 0.8654 - val_accuracy: 0.7461 - val_loss: 0.8439 - learning_rate: 0.0010
[INFO] Starting Stage 2 fine-tuning...
Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 189s 293ms/step - accuracy: 0.3031 - loss: 3.0502 - val_accuracy: 0.6630 - val_loss: 1.1642 - learning_rate: 1.0000e-05
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 186s 297ms/step - accuracy: 0.5482 - loss: 1.6563 - val_accuracy: 0.6737 - val_loss: 1.1333 - learning_rate: 1.0000e-05
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 191s 305ms/step - accuracy: 0.6262 - loss: 1.3194 - val_accuracy: 0.7055 - val_loss: 1.0096 - learning_rate: 1.0000e-05
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 185s 296ms/step - accuracy: 0.6704 - loss: 1.1415 - val_accuracy: 0.7286 - val_loss: 0.9151 - learning_rate: 1.0000e-05
Epoch 5/5
625/625 ━━━━━━━

✅ Model and history saved.
